In [1]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [2]:
import numpy as np
goods_image_vectors_embed = np.load('goods_image_vectors/goods_image_vectors/embed_deperson.npy')
goods_image_vectors_item = np.load('goods_image_vectors/goods_image_vectors/items_deperson.npy')


In [3]:
goods_title_vectors_embed = np.load('goods_title_vectors/goods_title_vectors/embed_deperson.npy')
goods_title_vectors_item = np.load('goods_title_vectors/goods_title_vectors/items_deperson.npy')

In [283]:
goods_title_vectors_item.shape

(1760568,)

In [4]:
offer_image_vectors_embed = np.load('offer_image_vectors/offer_image_vectors/embed_deperson.npy')
offer_image_vectors_item = np.load('offer_image_vectors/offer_image_vectors/items_deperson.npy')
offer_title_vectors_embed = np.load('offer_title_vectors/offer_title_vectors/embed_deperson.npy')
offer_title_vectors_item = np.load('offer_title_vectors/offer_title_vectors/items_deperson.npy')



In [285]:
offer_title_vectors_item

array(['477447', '95232', '117886', ..., '30926', '209577', '239103'],
      dtype='<U6')

переделаем в словарь

In [5]:
offer_image = {}
for key, value in zip(offer_image_vectors_item, offer_image_vectors_embed):
    offer_image[int(key)] = value
goods_image = {}
for key, value in zip(goods_image_vectors_item, goods_image_vectors_embed):
    goods_image[int(key)] = value

In [6]:
offer_title = {}
for key, value in zip(offer_title_vectors_item, offer_title_vectors_embed):
    offer_title[int(key)] = value
goods_title = {}
for key, value in zip(goods_title_vectors_item, goods_title_vectors_embed):
    goods_title[int(key)] = value

In [28]:
df_train

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,target,id
0,295140,1396793,37,0.027267,1070,NaN,1.400000e+01,0,295140$1396793
1,65291,1396586,38,0.050415,698,NaN,1.400000e+01,0,65291$1396586
2,39232,1396244,38,0.087280,837,NaN,1.400000e+01,0,39232$1396244
3,39232,1396513,38,0.087280,837,NaN,1.400000e+01,0,39232$1396513
4,65052,1396237,38,0.079773,1085,NaN,1.400000e+01,0,65052$1396237
...,...,...,...,...,...,...,...,...,...
2518436,464047,130193,6486,0.000057,559,NaN,2.463685e+15,0,464047$130193
2518437,464047,209576,6632,0.000043,559,692.0,2.463685e+15,0,464047$209576
2518438,4579,1163344,7503,0.000105,667,290.0,2.463685e+15,0,4579$1163344
2518439,340842,1163356,7597,0.000134,717,313.0,2.463685e+15,0,340842$1163356


In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps    

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [7]:
siamdataset = df_train[['offer_depersanalised','goods_depersanalised','target']].astype(int)

попытки  сделать сиамскую сеть и посчитать contrastive loss

In [134]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df, offer_title_vecs, goods_title_vecs):
        self.df = df
        self.offer_title_vecs = offer_title_vecs
        self.goods_title_vecs = goods_title_vecs

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        offer_idx = self.df['offer_depersanalised'][idx].tolist()
        goods_idx = self.df['goods_depersanalised'][idx].tolist()
        target = self.df['target'][idx].tolist()
        offer_vector = torch.FloatTensor(self.offer_title_vecs[offer_idx])
        goods_vector = torch.FloatTensor(self.goods_title_vecs[goods_idx])
        return offer_vector, goods_vector, target

custom_dataset = CustomDataset(siamdataset, offer_title_vectors_embed, goods_title_vectors_embed)
batch_size = 64
train_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [141]:
X_train, X_test = train_test_split(siamdataset, test_size=0.2, random_state=42, stratify=y)

In [163]:
X_train = X_train.reset_index(drop=True)


In [411]:
class SiameseNetwork(nn.Module):

    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(inplace=True),
            
            nn.Linear(128, 32),
            nn.ReLU(inplace=True),
            
            nn.Linear(32,2)
        )
        
    def forward_once(self, x):
        # This function will be called for both images
        # It's output is used to determine the similiarity
        output = self.fc1(x)
        return output

    def forward(self, input1, input2):
        # In this function we pass in both images and obtain both vectors
        # which are returned
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        return output1, output2

для эмбеддингов текстов и изображений

In [ ]:
class SiameseNetwork_2(nn.Module):

    def __init__(self):
        super(SiameseNetwork_2, self).__init__()

        # Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(
            nn.Linear(320, 128),
            nn.ReLU(inplace=True),
            
            nn.Linear(128, 32),
            nn.ReLU(inplace=True),
            
            nn.Linear(32,2)
        )
        
    def forward_once(self, x):
        # This function will be called for both images
        # It's output is used to determine the similiarity
        output = self.fc1(x)
        return output

    def forward(self, input1, input2):
        # In this function we pass in both images and obtain both vectors
        # which are returned
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        return output1, output2


In [145]:

class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
      # Calculate the euclidian distance and calculate the contrastive loss
      euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

      loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


      return loss_contrastive

In [171]:

custom_dataset = CustomDataset(X_train, offer_title_vectors_embed, goods_title_vectors_embed)
batch_size = 64
train_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)
net_text = SiameseNetwork()
criterion = ContrastiveLoss()
optimizer = optim.Adam(net_text.parameters(), lr = 0.0005 )


In [ ]:
counter = []
loss_history = [] 
iteration_number= 0
net_text = SiameseNetwork()
criterion = ContrastiveLoss()
optimizer = optim.Adam(net_text.parameters(), lr = 0.0005 )

# Iterate throught the epochs
for epoch in tqdm(range(10)):

    # Iterate over batches
    for i in range(len(siamdataset)):
        offer_idx = siamdataset['offer_depersanalised'][i]
        goods_idx = siamdataset['goods_depersanalised'][i]
        target = siamdataset['target'][i]
        v1 = torch.FloatTensor(offer_title[offer_idx])
        v2 = torch.FloatTensor(goods_title[goods_idx])
        # Send the images and labels to CUDA

        # Zero the gradients
        optimizer.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = net_text(v1, v2)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = criterion(output1, output2, label)

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()

        # Every 10 batches print out the loss
        if i % 10 == 0 :
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 10

            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())
text_contrastive_loss =[]
for i in range(len(siamdataset)):
    offer_idx = siamdataset['offer_depersanalised'][i]
    goods_idx = siamdataset['goods_depersanalised'][i]
    v1 = torch.FloatTensor(offer_title[offer_idx])
    v2 = torch.FloatTensor(goods_title[goods_idx])
    output1, output2 = net_text(v1, v2)
    text_contrastive_loss.append(F.pairwise_distance(output1, output2).detach().numpy()[0])
siamdataset['text_contrastive_loss']=text_contrastive_loss

для изображений 

In [ ]:
counter = []
loss_history = [] 
iteration_number= 0
net_image = SiameseNetwork_2()
criterion = ContrastiveLoss()
optimizer = optim.Adam(net_image.parameters(), lr = 0.0005 )

# Iterate throught the epochs
for epoch in tqdm(range(10)):

    # Iterate over batches
    for i in range(len(siamdataset)):
        offer_idx = siamdataset['offer_depersanalised'][i]
        goods_idx = siamdataset['goods_depersanalised'][i]
        if offer_idx in offer_image_items and goods_idx in goods_image_items:
            target = siamdataset['target'][i]
            offer_vector_title = offer_title[offer_idx]
            goods_vector_title = goods_title[goods_idx]
            offer_vector_image = offer_image[offer_idx]
            goods_vector_image = goods_image[goods_idx]
            offer = np.concatenate((offer_vector_title, offer_vector_image), axis=0)
            goods = np.concatenate((goods_vector_title, goods_vector_image), axis=0)
            v1 = torch.FloatTensor(offer)
            v2 = torch.FloatTensor(goods)
            # Send the images and labels to CUDA
    
            # Zero the gradients
            optimizer.zero_grad()
    
            # Pass in the two images into the network and obtain two outputs
            output1, output2 = net_image(v1, v2)
    
            # Pass the outputs of the networks and label into the loss function
            loss_contrastive = criterion(output1, output2, label)
    
            # Calculate the backpropagation
            loss_contrastive.backward()
    
            # Optimize
            optimizer.step()
    
            # Every 10 batches print out the loss
            if i % 10 == 0 :
                print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
                iteration_number += 10
    
                counter.append(iteration_number)
                loss_history.append(loss_contrastive.item())
image_contrastive_loss =[]
for i in range(len(siamdataset)):
        offer_idx = siamdataset['offer_depersanalised'][i]
        goods_idx = siamdataset['goods_depersanalised'][i]
        if offer_idx in offer_image_items and goods_idx in goods_image_items:
            target = siamdataset['target'][i]
            offer_vector_title = offer_title[offer_idx]
            goods_vector_title = goods_title[goods_idx]
            offer_vector_image = offer_image[offer_idx]
            goods_vector_image = goods_image[goods_idx]
            offer = np.concatenate((offer_vector_title, offer_vector_image), axis=0)
            goods = np.concatenate((goods_vector_title, goods_vector_image), axis=0)
            v1 = torch.FloatTensor(offer)
            v2 = torch.FloatTensor(goods)
          
            output1, output2 = net_image(v1, v2)
            image_contrastive_loss.append(F.pairwise_distance(output1, output2).detach().numpy()[0])
        else:
            image_contrastive_loss.append(0)
siamdataset['image_contrastive_loss']=image_contrastive_loss

  0%|                                                    | 0/10 [00:00<?, ?it/s]

Epoch number 0
 Current loss 0.01679709553718567

Epoch number 0
 Current loss 0.021385129541158676

Epoch number 0
 Current loss 0.06412968039512634

Epoch number 0
 Current loss 0.009067622013390064

Epoch number 0
 Current loss 0.006499303039163351

Epoch number 0
 Current loss 0.002301202155649662

Epoch number 0
 Current loss 0.014159534126520157

Epoch number 0
 Current loss 0.0018176493467763066

Epoch number 0
 Current loss 0.003790738293901086

Epoch number 0
 Current loss 0.00483170710504055

Epoch number 0
 Current loss 0.004374033771455288

Epoch number 0
 Current loss 0.0031995484605431557

Epoch number 0
 Current loss 0.011018787510693073

Epoch number 0
 Current loss 0.00026922288816422224

Epoch number 0
 Current loss 0.0012276816414669156

Epoch number 0
 Current loss 0.007446851581335068

Epoch number 0
 Current loss 0.005719807930290699

Epoch number 0
 Current loss 0.0013655395014211535

Epoch number 0
 Current loss 0.0005082960124127567

Epoch number 0
 Current los

In [241]:
test_dataset = CustomDataset(siamdataset, offer_title_vectors_embed, goods_title_vectors_embed)
batch_size = 10000
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

шалость не удалась(

идем проще, считаем всевозможные расстояние между вектораами, потом в бустинг

In [9]:
from scipy.spatial import distance
offer_image_items = offer_image.keys()
goods_image_items = goods_image.keys()
from tqdm import tqdm
manhattan_distances_t = []
euclidean_distances_t= []
cosine_distances_t = []
chebyshev_distances_t = []
canberra_distances_t = []
for i in tqdm(range(len(siamdataset))):
    offer_idx = siamdataset['offer_depersanalised'][i]
    goods_idx = siamdataset['goods_depersanalised'][i]
    
    if offer_idx in offer_image_items and goods_idx in goods_image_items:
        offer_vector_title = offer_title[offer_idx]
        goods_vector_title = goods_title[goods_idx]
        offer_vector_image = offer_image[offer_idx]
        goods_vector_image = goods_image[goods_idx]
        v1 = np.concatenate((offer_vector_title, offer_vector_image), axis=0)
        v2 = np.concatenate((goods_vector_title, goods_vector_image), axis=0)
        manhattan_distances_t.append(np.sum(np.abs(v1 - v2)))
        euclidean_distances_t.append(np.linalg.norm(v1 - v2))
        cosine_distances_t.append(1 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        chebyshev_distances_t.append(np.max(np.abs(v1 - v2)))
        canberra_distances_t.append(np.sum(np.abs(v1 - v2) / (np.abs(v1) + np.abs(v2))))
    else:
        manhattan_distances_t.append(0)
        euclidean_distances_t.append(0)
        cosine_distances_t.append(0)
        chebyshev_distances_t.append(0)
        canberra_distances_t.append(0)
        
siamdataset['manhattan_distances_t'] = manhattan_distances_t
siamdataset['euclidean_distances_t'] = euclidean_distances_t
siamdataset['cosine_distances_t'] = cosine_distances_t
siamdataset['chebyshev_distances_t'] = chebyshev_distances_t
siamdataset['canberra_distances_t'] = canberra_distances_t

100%|██████████████████████████████| 2518441/2518441 [00:47<00:00, 52559.25it/s]


считаем метрики только для текста

In [10]:
from scipy.spatial import distance
offer_image_items = offer_image.keys()
goods_image_items = goods_image.keys()
from tqdm import tqdm
manhattan_distances_t = []
euclidean_distances_t= []
cosine_distances_t = []
chebyshev_distances_t = []
pearson_distances_t = []
canberra_distances_t = []
for i in tqdm(range(len(siamdataset))):
    offer_idx = siamdataset['offer_depersanalised'][i]
    goods_idx = siamdataset['goods_depersanalised'][i]
    
    v1 = offer_title[offer_idx]
    v2 = goods_title[goods_idx]
    manhattan_distances_t.append(np.sum(np.abs(v1 - v2)))
    euclidean_distances_t.append(np.linalg.norm(v1 - v2))
    cosine_distances_t.append(1 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    chebyshev_distances_t.append(np.max(np.abs(v1 - v2)))
    canberra_distances_t.append(np.sum(np.abs(v1 - v2) / (np.abs(v1) + np.abs(v2))))

  
siamdataset['manhattan_distances_text'] = manhattan_distances_t
siamdataset['euclidean_distances_text'] = euclidean_distances_t
siamdataset['cosine_distances_text'] = cosine_distances_t
siamdataset['chebyshev_distances_text'] = chebyshev_distances_t
siamdataset['canberra_distances_text'] = canberra_distances_t


100%|██████████████████████████████| 2518441/2518441 [01:52<00:00, 22403.41it/s]


In [11]:
X = siamdataset[['manhattan_distances_t','euclidean_distances_t','cosine_distances_t','chebyshev_distances_t','canberra_distances_t','manhattan_distances_text','euclidean_distances_text','cosine_distances_text','chebyshev_distances_text','canberra_distances_text']]
y = siamdataset[['target']]

In [12]:
X[['goods_category_id','attrs+title_score','sum_length']] = df_train[['goods_category_id','attrs+title_score','sum_length']]

/tmp/ipykernel_264873/832130725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['goods_category_id','attrs+title_score','sum_length']] = df_train[['goods_category_id','attrs+title_score','sum_length']]
/tmp/ipykernel_264873/832130725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['goods_category_id','attrs+title_score','sum_length']] = df_train[['goods_category_id','attrs+title_score','sum_length']]


In [62]:
X[['offer_price','goods_price']] = df_train[['offer_price','goods_price']]
X = X.fillna(1)
X['diff'] = X['offer_price'] /X['goods_price'] 

In [20]:
X = X.fillna(1)

In [72]:
X['diff'] = X['offer_price'] /X['goods_price'] 

In [344]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 5.7 MB/s eta 0:00:0000:0100:01


In [132]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Разделение данных на тренировочный и тестовый набор
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,)

# Создание и обучение модели XGBoost
params = {
    'objective': 'binary:logistic',
    'max_depth': 13,
    'learning_rate': 0.07,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'min_child_weight': 2,
    'gamma': 0.1,
    'lambda': 0,
    'alpha': 0,
}
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

model = xgb.train(params, dtrain, num_boost_round=200)


y_pred_proba = model.predict(dtest)
threshold = 0.5
y_pred_class = [1 if p > threshold else 0 for p in y_pred_proba]

# Рассчет F1 Score (macro)
f1_macro = f1_score(y_test, y_pred_class, average='macro')

print(f'F1 Score (macro) модели XGBoost: {f1_macro}')


F1 Score (macro) модели XGBoost: 0.9217690699653027


In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)
from sklearn.tree import DecisionTreeClassifier
# Создание и обучение модели XGBoost
#model = xgb.XGBClassifier()
#model.fit(X_train, y_train)

tree_model = DecisionTreeClassifier()

# Обучение модели на тренировочных данных
tree_model.fit(X_train, y_train)

# Предсказание классов на тестовых данных
y_pred = tree_model.predict(X_test)

# Рассчет F1 Score (macro)
f1_macro = f1_score(y_test, y_pred, average='macro')

print(f'F1 Score (macro) модели XGBoost: {f1_macro}')

F1 Score (macro) модели XGBoost: 0.879561680871481


In [ ]:
model.fit(X_train, y_train)

In [45]:
dtrain = xgb.DMatrix(X, label=y)
dtest = xgb.DMatrix(X_test, label=y_test)

model = xgb.train(params, dtrain, num_boost_round=100)

In [43]:
y_pred_proba

array([5.52287474e-02, 1.07805135e-04, 8.55661288e-04, ...,
       1.07007181e-04, 1.20749552e-04, 8.77032362e-05], dtype=float32)

In [32]:
X_result = df_test[['offer_depersanalised','goods_depersanalised']].astype(int)

In [33]:
from scipy.spatial import distance
offer_image_items = offer_image.keys()
goods_image_items = goods_image.keys()
from tqdm import tqdm
manhattan_distances_t = []
euclidean_distances_t= []
cosine_distances_t = []
chebyshev_distances_t = []
canberra_distances_t = []
for i in tqdm(range(len(X_result))):
    offer_idx = X_result['offer_depersanalised'][i]
    goods_idx = X_result['goods_depersanalised'][i]
    
    if offer_idx in offer_image_items and goods_idx in goods_image_items:
        offer_vector_title = offer_title[offer_idx]
        goods_vector_title = goods_title[goods_idx]
        offer_vector_image = offer_image[offer_idx]
        goods_vector_image = goods_image[goods_idx]
        v1 = np.concatenate((offer_vector_title, offer_vector_image), axis=0)
        v2 = np.concatenate((goods_vector_title, goods_vector_image), axis=0)
        manhattan_distances_t.append(np.sum(np.abs(v1 - v2)))
        euclidean_distances_t.append(np.linalg.norm(v1 - v2))
        cosine_distances_t.append(1 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        chebyshev_distances_t.append(np.max(np.abs(v1 - v2)))
        canberra_distances_t.append(np.sum(np.abs(v1 - v2) / (np.abs(v1) + np.abs(v2))))
    else:
        manhattan_distances_t.append(0)
        euclidean_distances_t.append(0)
        cosine_distances_t.append(0)
        chebyshev_distances_t.append(0)
        canberra_distances_t.append(0)
        
X_result['manhattan_distances_t'] = manhattan_distances_t
X_result['euclidean_distances_t'] = euclidean_distances_t
X_result['cosine_distances_t'] = cosine_distances_t
X_result['chebyshev_distances_t'] = chebyshev_distances_t
X_result['canberra_distances_t'] = canberra_distances_t

from scipy.spatial import distance
offer_image_items = offer_image.keys()
goods_image_items = goods_image.keys()
from tqdm import tqdm
manhattan_distances_t = []
euclidean_distances_t= []
cosine_distances_t = []
chebyshev_distances_t = []
pearson_distances_t = []
canberra_distances_t = []
for i in tqdm(range(len(X_result))):
    offer_idx = X_result['offer_depersanalised'][i]
    goods_idx = X_result['goods_depersanalised'][i]
    
    v1 = offer_title[offer_idx]
    v2 = goods_title[goods_idx]
    manhattan_distances_t.append(np.sum(np.abs(v1 - v2)))
    euclidean_distances_t.append(np.linalg.norm(v1 - v2))
    cosine_distances_t.append(1 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    chebyshev_distances_t.append(np.max(np.abs(v1 - v2)))
    canberra_distances_t.append(np.sum(np.abs(v1 - v2) / (np.abs(v1) + np.abs(v2))))

  
X_result['manhattan_distances_text'] = manhattan_distances_t
X_result['euclidean_distances_text'] = euclidean_distances_t
X_result['cosine_distances_text'] = cosine_distances_t
X_result['chebyshev_distances_text'] = chebyshev_distances_t
X_result['canberra_distances_text'] = canberra_distances_t


100%|████████████████████████████████| 363835/363835 [00:15<00:00, 22810.14it/s]


In [34]:
X_result[['goods_category_id','attrs+title_score','sum_length']] = df_test[['goods_category_id','attrs+title_score','sum_length']]

In [35]:
X_t = X_result[['manhattan_distances_t','euclidean_distances_t','cosine_distances_t','chebyshev_distances_t','canberra_distances_t','manhattan_distances_text','euclidean_distances_text','cosine_distances_text','chebyshev_distances_text','canberra_distances_text',
                                  'goods_category_id','attrs+title_score','sum_length']]

In [36]:
X_t = X_t.fillna(1)

In [64]:
X_t[['offer_price','goods_price']] = df_test[['offer_price','goods_price']]
X_t = X_t.fillna(1)
X_t['diff'] = X_t['offer_price'] /X_t['goods_price']

In [86]:
y_score = model.predict(X_t)

In [133]:
dtest = xgb.DMatrix(X_t)
y_pred_proba = model.predict(dtest)
threshold = 0.5
y_pred_class = [1 if p > threshold else 0 for p in y_pred_proba]

In [134]:
sum(y_pred_class)

40798

In [135]:
df_test['target'] = y_pred_class

In [379]:
df_test

,offer_depersanalised,goods_depersanalised,sum_length,attrs+title_score,offer_price,goods_price,goods_category_id,id,target
0,64819,1396468,38,0.046997,368,NaN,1.400000e+01,64819$1396468,0
1,64819,1396235,38,0.046997,368,NaN,1.400000e+01,64819$1396235,0
2,64819,1396318,38,0.046997,368,NaN,1.400000e+01,64819$1396318,0
3,359959,1396281,40,0.060211,634,NaN,1.400000e+01,359959$1396281,0
4,142700,717657,40,0.000370,14924,31840.0,2.000000e+00,142700$717657,0
...,...,...,...,...,...,...,...,...,...
363830,122775,310950,5061,0.000036,804,NaN,2.463685e+15,122775$310950,0
363831,419632,342465,5184,0.000092,332,184.0,2.463685e+15,419632$342465,0
363832,369393,130129,5447,0.000267,660,614.0,2.463685e+15,369393$130129,0
363833,24514,130142,5760,0.001467,2000,259.0,2.463685e+15,24514$130142,0


In [136]:
submis = df_test[['id','target']]

In [137]:
submis.to_csv('submis2.csv',index=False)

In [ ]:
а